# Classify Images using SqueezeNet

## Import Turi Create
Please follow the repository README instructions to install the Turi Create package.

**Note**: Turi Create is currently only compatible with Python 2.7

In [1]:
import turicreate as turi

## Reference the dataset path

In [2]:
url = "data/food_images"

## Label the dataset
In the following block of code we will labels the image in the dataset of **Egg** and **Soup** images. Then we will export it as an `SFrame` data object to use it for training the image classification model.

1. The first line of code loads the folder images content using the `image_analysis` property. 

2. The second line creates a _foodType_ key for each image in the dataset to specify whether it's an **Egg** or **Soup** based on which folder it's located in.

3. The third line exports the analyzed data as an `SFrame` object in order to use it while creating our image classifier.

4. The fourth line simply visualises the new labeled image into a large list.

**Note**:- You do not have to run the following block of code everytime you create a classifer, unless you changed/edited the dataset.

In [3]:
data = turi.image_analysis.load_images(url)
data["foodType"] = data["path"].apply(lambda path: "Eggs" if "eggs" in path else "Soup")
data.save("egg_or_soup.sframe")
data.explore()

Unsupported image format. Supported formats are JPEG and PNG	 file: /Users/ahmedbekhit/Documents/Data/Development/TuriCreate/repo/turicreate-notebook/notebooks/data/food_images/eggs/.DS_Store

Unsupported image format. Supported formats are JPEG and PNG	 file: /Users/ahmedbekhit/Documents/Data/Development/TuriCreate/repo/turicreate-notebook/notebooks/data/food_images/.DS_Store

Unsupported image format. Supported formats are JPEG and PNG	 file: /Users/ahmedbekhit/Documents/Data/Development/TuriCreate/repo/turicreate-notebook/notebooks/data/food_images/soup/.DS_Store

Materializing SFrame...

Done.

## Load the labeled SFrame
In the following line of code we are loading the `SFrame` object that contains the images in our dataset with their labels.

In [4]:
dataBuffer = turi.SFrame("egg_or_soup.sframe")

## Create training and test data using our existing dataset
Here, we're randomly splitting the data.
- 90% of the data in the `SFrame` object will be used for training the image classifier.
- 10% of the data in the `SFrame` object will be used for testing the image classifier.

In [5]:
trainingBuffers, testingBuffers = dataBuffer.random_split(0.9)

## Train the image classifier
In the following line of code, we will create an image classifier and we'll feed it with the training data we have. 

In this example, the image classifer's architecture will be using SqueezNet that usually reduces the size of your exported model.

Check out the official paper here: https://arxiv.org/pdf/1602.07360.pdf.

In [6]:
model = turi.image_classifier.create(trainingBuffers, target="foodType", model="squeezenet_v1.1")

Download completed: /var/tmp/model_cache/squeezenet_v1.1-symbol.json
Download completed: /var/tmp/model_cache/squeezenet_v1.1-0000.params
Resizing images...
Performing feature extraction on resized images...
Completed 269/269
PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 254

...

| 5         | 13       | 1.000000  | 1.264025     | 0.685039          | 0.466667            |

| 6         | 14       | 1.000000  | 1.297765     | 0.496063          | 0.733333            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

## Evaluate the test data to determine the model accuracy

In [7]:
evaluations = model.evaluate(testingBuffers)
print evaluations["accuracy"]

0.903225806452


## Save the Turi Create model to retrieve it later

In [8]:
model.save("egg_or_soup.model")

## Export the image classification model for Core ML

In [9]:
model.export_coreml("EggSoupClassifier.mlmodel")